In [ ]:
# Imports
import numpy as np
from lda import LDA
from bayes import bayes_nonparametric
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load data
trainDataNP = np.load("fashion_train.npy")
testDataNP = np.load("fashion_test.npy")

# Split data into X and y arrays
X_train = trainDataNP[:, :-1]
y_train = trainDataNP[:, -1]
X_test = testDataNP[:, :-1]
y_test = testDataNP[:, -1]

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Run through LDA
lda = LDA(k = 4)
X_projected = lda.fit_transform(X_train_scaled, y_train)

In [ ]:
bayes = bayes_nonparametric()
bayes.train(X_projected, y_train, h = .75)
y_pred = bayes.predict(X_test_scaled)

In [ ]:
arr1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

# for col in arr1:
#     print(row)
#     print('next')

dict1 = {'a': 1, 'b': 2}

dict1['a'] *= 100

dict1
